# Model Initialization

> Utilities for initializing and configuring models for rank manipulation experiments

In [1]:
#| default_exp models.init_utils

In [2]:
#| hide
try:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/rank-bert
except:
    pass

Mounted at /content/drive
/content/drive/MyDrive/rank-bert


In [3]:
#| hide
try:
    !pip install -q nbdev
except:
    pass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.7 MB/s eta 0:00:00


In [4]:
#| hide
from nbdev.showdoc import *

In [32]:
#| export
import torch
import torch.nn as nn
import math
import numpy as np
from copy import deepcopy
from typing import Dict, List, Optional, Tuple, Union

from fastai.text.all import *
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    BertConfig,
    BertForSequenceClassification,
    BertModel
)

from rank_bert.models.base_models import (
    get_pretrained_model,
    BertWrapper,
    count_parameters,
    BERT_CONFIGS,
    GLUE_NUM_LABELS
)

## Advanced Model Initialization Utilities

This module provides advanced utilities for initializing and configuring BERT models for rank manipulation experiments. These utilities extend the basic model creation functions from the base_models module.

In [6]:
#| export
def create_model_from_config(config: Union[Dict, BertConfig], model_class=BertForSequenceClassification):
    """
    Create a model from a configuration dictionary or BertConfig object.

    Args:
        config (Dict or BertConfig): Configuration for the model
        model_class: The model class to instantiate (default: BertForSequenceClassification)

    Returns:
        The instantiated model
    """
    if isinstance(config, dict):
        config = BertConfig(**config)

    return model_class(config)

In [7]:
#| export
def get_model_config(model_name: str, task_name: str, num_labels: Optional[int] = None) -> BertConfig:
    """
    Get a configuration object for a model.

    Args:
        model_name (str): HuggingFace model name or path (e.g., 'prajjwal1/bert-tiny')
        task_name (str): GLUE task name ('sst2', 'mrpc', 'rte')
        num_labels (int, optional): Number of output labels

    Returns:
        BertConfig: Configuration object for the model
    """
    # Get the number of labels for the task
    num_labels = num_labels or GLUE_NUM_LABELS.get(task_name, 2)

    # Check if the model name is a known configuration or a HuggingFace model
    if model_name in BERT_CONFIGS:
        # Create a new configuration with the specified parameters
        config = BertConfig(
            **BERT_CONFIGS[model_name],
            num_labels=num_labels,
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.1
        )
    else:
        # Load a pretrained configuration from HuggingFace
        config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)

    return config

In [33]:
#| export
def modify_config_for_rank_experiments(config: BertConfig, hidden_size: Optional[int] = None,
                                      num_hidden_layers: Optional[int] = None,
                                      num_attention_heads: Optional[int] = None,
                                      intermediate_size: Optional[int] = None) -> BertConfig:
    """
    Modify a configuration for rank manipulation experiments.

    Args:
        config (BertConfig): Original configuration
        hidden_size (int, optional): Hidden size for the model
        num_hidden_layers (int, optional): Number of hidden layers
        num_attention_heads (int, optional): Number of attention heads
        intermediate_size (int, optional): Intermediate size for feed-forward networks

    Returns:
        BertConfig: Modified configuration
    """
    # Create a copy of the config to avoid modifying the original
    new_config = deepcopy(config)

    # Update configuration parameters if provided
    if hidden_size is not None:
        new_config.hidden_size = hidden_size

    if num_hidden_layers is not None:
        new_config.num_hidden_layers = num_hidden_layers

    if num_attention_heads is not None:
        new_config.num_attention_heads = num_attention_heads

    if intermediate_size is not None:
        new_config.intermediate_size = intermediate_size

    return new_config

In [34]:
#| export
def initialize_weights(model: nn.Module, method: str = 'normal', **kwargs):
    """
    Initialize weights of a model using the specified method.

    Args:
        model (nn.Module): PyTorch model
        method (str): Initialization method ('normal', 'xavier', 'kaiming', 'orthogonal')
        **kwargs: Additional arguments for the initialization method

    Returns:
        nn.Module: Model with initialized weights
    """
    def _init_weights(module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            # Initialize weights based on the specified method
            if method == 'normal':
                std = kwargs.get('std', 0.02)
                nn.init.normal_(module.weight, mean=0.0, std=std)
            elif method == 'xavier':
                gain = kwargs.get('gain', 1.0)
                nn.init.xavier_normal_(module.weight, gain=gain)
            elif method == 'kaiming':
                a = kwargs.get('a', 0)
                mode = kwargs.get('mode', 'fan_in')
                nonlinearity = kwargs.get('nonlinearity', 'relu')
                nn.init.kaiming_normal_(module.weight, a=a, mode=mode, nonlinearity=nonlinearity)
            elif method == 'orthogonal':
                gain = kwargs.get('gain', 1.0)
                nn.init.orthogonal_(module.weight, gain=gain)
            else:
                raise ValueError(f"Unsupported initialization method: {method}")

            # Initialize bias if it exists
            if hasattr(module, 'bias') and module.bias is not None:
                nn.init.zeros_(module.bias)

        elif isinstance(module, nn.LayerNorm):
            # LayerNorm layers are initialized with ones and zeros
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)

    # Apply initialization to all submodules
    model.apply(_init_weights)

    return model

In [35]:
#| export
def default_transformer_splitter(model):
    """
    Default parameter group splitter for transformer models.
    Separates parameters into layer-wise groups for differential learning rates.

    Args:
        model: Transformer model

    Returns:
        List of parameter groups
    """
    # Get the actual model if it's wrapped
    if hasattr(model, 'model'):
        model = model.model

    # Extract layers from the model
    embeddings = [p for n, p in model.named_parameters() if 'embeddings' in n]
    encoder_layers = []

    # Check if the model has a bert attribute (BertForSequenceClassification)
    if hasattr(model, 'bert'):
        num_layers = model.bert.config.num_hidden_layers
        # Collect parameters for each encoder layer
        for i in range(num_layers):
            encoder_layers.append([p for n, p in model.named_parameters()
                                  if f'encoder.layer.{i}.' in n])
    # Handle base BERT model
    elif hasattr(model, 'encoder'):
        num_layers = model.config.num_hidden_layers
        # Collect parameters for each encoder layer
        for i in range(num_layers):
            encoder_layers.append([p for n, p in model.named_parameters()
                                  if f'encoder.layer.{i}.' in n])

    # Extract classifier parameters
    classifier = [p for n, p in model.named_parameters() if 'classifier' in n or 'pooler' in n]

    # Construct parameter groups, reverting order for discriminative learning rates
    # (later layers with higher learning rates)
    param_groups = encoder_layers + [classifier, embeddings]
    param_groups.reverse()

    return param_groups

In [36]:
#| export
class TransformerCallback(Callback):
    """
    Callback for handling transformer-specific training operations.
    Manages input/output format conversion between fastai and transformers.
    """

    def __init__(self):
        self.tokenizer = None

    def before_batch(self):
        "Ensure inputs are in the expected format for transformers"
        # fastai might pack the inputs from the data loader
        if isinstance(self.learn.xb, tuple) and len(self.learn.xb) == 1:
            self.learn.xb = self.learn.xb[0]

    def after_pred(self):
        "Process outputs from the model"
        # If the model returns a tuple or list (common for transformers), handle it
        if isinstance(self.learn.pred, tuple) or isinstance(self.learn.pred, list):
            self.learn.pred = self.learn.pred[0]  # Usually logits are the first element

In [37]:
#| export
def create_model_with_config(config_or_model_name, task_name=None, num_labels=None, **kwargs):
    """
    Create a model using either a configuration object or model name.

    Args:
        config_or_model_name: Either a BertConfig object or a model name string
        task_name (str, optional): GLUE task name (required if config_or_model_name is a string)
        num_labels (int, optional): Number of output labels
        **kwargs: Additional arguments to modify the configuration

    Returns:
        The instantiated model
    """
    if isinstance(config_or_model_name, (BertConfig, dict)):
        config = config_or_model_name
        if isinstance(config, dict):
            if num_labels is not None:
                config['num_labels'] = num_labels
            config = BertConfig(**config)
        elif num_labels is not None:
            config.num_labels = num_labels
    else:
        # Assume config_or_model_name is a model name string
        if task_name is None:
            raise ValueError("task_name must be provided when config_or_model_name is a model name string")
        config = get_model_config(config_or_model_name, task_name, num_labels)

    # Apply configuration modifications
    if kwargs:
        config = modify_config_for_rank_experiments(config, **kwargs)

    return create_model_from_config(config)

In [38]:
#| export
def get_custom_bert_model(model_type, task_name, num_labels=None):
    """
    Get a customized BERT model for rank experiments.

    Args:
        model_type (str): Type of BERT model ('tiny', 'mini', 'small', 'custom')
        task_name (str): GLUE task name
        num_labels (int, optional): Number of output labels

    Returns:
        The instantiated model
    """
    # Map model type to HuggingFace model name
    model_mapping = {
        'tiny': 'prajjwal1/bert-tiny',
        'mini': 'prajjwal1/bert-mini',
        'small': 'prajjwal1/bert-small'
    }

    if model_type in model_mapping:
        return get_pretrained_model(model_mapping[model_type], task_name, num_labels)
    elif model_type == 'custom':
        # Create a fully custom BERT model with specified parameters
        config = BertConfig(
            vocab_size=30522,  # Default BERT vocab size
            hidden_size=256,
            num_hidden_layers=4,
            num_attention_heads=4,
            intermediate_size=1024,
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.1,
            max_position_embeddings=512,
            type_vocab_size=2,
            initializer_range=0.02,
            layer_norm_eps=1e-12,
            num_labels=num_labels or GLUE_NUM_LABELS.get(task_name, 2)
        )
        return BertForSequenceClassification(config)
    else:
        raise ValueError(f"Unknown model type: {model_type}. Use 'tiny', 'mini', 'small', or 'custom'")

In [39]:
#| export
def optimize_model_for_training(model, device=None, mixed_precision=False):
    """
    Optimize a model for efficient training.

    Args:
        model: PyTorch model
        device: Device to move the model to (default: detect automatically)
        mixed_precision: Whether to use mixed precision training

    Returns:
        Optimized model
    """
    # Determine device if not provided
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Move model to appropriate device
    model = model.to(device)

    # Apply mixed precision if requested and on GPU
    if mixed_precision and device.type == 'cuda':
        try:
            from torch.cuda.amp import autocast
            # Wrap model's forward method with autocast
            original_forward = model.forward
            model.forward = autocast()(original_forward)
        except ImportError:
            print("Mixed precision training not available. Falling back to full precision.")

    # Free unused memory
    if device.type == 'cuda':
        torch.cuda.empty_cache()

    return model

## Examples

Here are some examples of how to use the model initialization utilities.

In [40]:
# Example 1: Create a model with custom configuration
custom_config = get_model_config('prajjwal1/bert-mini', 'sst2')
custom_config = modify_config_for_rank_experiments(custom_config, hidden_size=384, num_attention_heads=6)
model = create_model_from_config(custom_config)
print(f"Custom model parameter count: {count_parameters(model):,}")

Custom model parameter count: 17,590,146


In [41]:
# Example 2: Create a model and initialize its weights using orthogonal initialization
model = get_custom_bert_model('tiny', 'mrpc')
model = initialize_weights(model, method='orthogonal', gain=1.0)
print(f"Orthogonally initialized model parameter count: {count_parameters(model):,}")

Orthogonally initialized model parameter count: 4,386,178


In [42]:
# Example 3: Use the simplified model creation function
model = create_model_with_config('prajjwal1/bert-mini', 'rte', hidden_size=320, num_attention_heads=5)
print(f"Custom model parameter count: {count_parameters(model):,}")

Custom model parameter count: 14,310,978


In [43]:
#| hide
import nbdev; nbdev.nbdev_export()